In [35]:
import os
import random
import pickle
import matplotlib.pyplot as plt
import random
import json

from termcolor import cprint 

from pycolonies import Colonies
from pycolonies import colonies_client
from pycolonies import func_spec
from pycolonies import Workflow

colonies, colonyid, colony_prvkey, executorid, executor_prvkey = colonies_client()

In [4]:
!colonies fs sync -l src -d /root/cfs/src --yes

 Uploading classifier-checkpoint.py 100% [===============] (24 MB/s)


In [37]:
src_snapshotname = str(random.randint(0,9**60))
src_snapshot = colonies.create_snapshot(colonyid, "/src", src_snapshotname, executor_prvkey)
data_snapshotname = str(random.randint(0,9**60))
data_snapshot = colonies.create_snapshot(colonyid, "/data", data_snapshotname, executor_prvkey)

def train_nn(ll):
   return {
      "conditions": {
          "colonyid": colonyid,
          "executortype": "borg-hpcexecutor",
          #"executortype": "leonardo-hpcexecutor",
          "nodes": 1,
          "processes_per_node": 1,
          #"mem": "30G",
          "walltime": 600,
          "gpu": {
              "count": 1
          }
      },
      "funcname": "execute",
      "kwargs": {
          "cmd": "python3",
          "docker-image": "johan/classifier",
          "rebuild-image": True,
          "args": [
              "/cfs/src/{processid}/classifier.py"
          ],
          "learning-rate": ll
      },
      "fs": {
          "mount": "/cfs",
          "snapshots": [
              {
                  "snapshotid": src_snapshot["snapshotid"],
                  "label": "/src",
                  "dir": "/src/{processid}",
                  "keepfiles": True,
                  "keepsnaphot": True
              },
              {
                  "snapshotid": data_snapshot["snapshotid"],
                  "label": "/data",
                  "dir": "/data",
                  "keepfiles": True,
                  "keepsnaphot": False
              }
          ],
          "dirs": [
              {
                  "label": "/results",
                  "dir": "/results",
                  "keepfiles": True,
                  "onconflicts": {
                      "onstart": {
                          "keeplocal": False
                      },
                      "onclose": {
                          "keeplocal": True
                      }
                  }
              }
          ]
      },
      "maxwaittime": -1,
      "maxexectime": 600,
      "maxretries": 3
  }

src_snapshotname_eval_ll = str(random.randint(0,9**60))
src_snapshot_eval_ll = colonies.create_snapshot(colonyid, "/src", src_snapshotname_eval_ll, executor_prvkey)

def eval_ll():
   return {
      "conditions": {
          "colonyid": colonyid,
          "executortype": "borg-hpcexecutor",
          "nodes": 1,
          "processes_per_node": 1,
          "walltime": 600,
          "gpu": {
              "count": 1
          }
      },
      "funcname": "execute",
      "kwargs": {
          "cmd": "python3",
          "docker-image": "johan/classifier",
          "rebuild-image": False,
          "args": [
              "/cfs/src/{processid}/eval_ll.py"
          ]
      },
       "fs": {
          "mount": "/cfs",
          "snapshots": [
              {
                  "snapshotid": src_snapshot["snapshotid"],
                  "label": "/src",
                  "dir": "/src/{processid}",
                  "keepfiles": False,
                  "keepsnaphot": False
              }
          ]
      },
      "maxwaittime": -1,
      "maxexectime": 600,
      "maxretries": 3
  }

wf = Workflow(colonyid)
wf.add(train_nn(ll=0.0001), nodename="nn_ll_0.0001", dependencies=[])
#wf.add(train_nn(ll=0.0002), nodename="nn_ll_0.0002", dependencies=[])
#wf.add(eval_ll(), nodename="eval", dependencies=["nn_ll_0.0001", "nn_ll_0.0002"])
wf.add(eval_ll(), nodename="eval", dependencies=["nn_ll_0.0001"])

print(train_nn(ll=0.0001))
processgraph = colonies.submit(wf, executor_prvkey)

{'conditions': {'colonyid': '0f7eb1ddfada13889e3570bf321d59c327d1ec3d7a94a5df1f710a2395fcfbf8', 'executortype': 'borg-hpcexecutor', 'nodes': 1, 'processes_per_node': 1, 'walltime': 600, 'gpu': {'count': 1}}, 'funcname': 'execute', 'kwargs': {'cmd': 'python3', 'docker-image': 'johan/classifier', 'rebuild-image': True, 'args': ['/cfs/src/{processid}/classifier.py'], 'learning-rate': 0.0001}, 'fs': {'mount': '/cfs', 'snapshots': [{'snapshotid': '2d71eb95c14b10c455c9f391bc06389e61ea6a6422116b577e3ce9400c1acb3a', 'label': '/src', 'dir': '/src/{processid}', 'keepfiles': True, 'keepsnaphot': True}, {'snapshotid': '781358b7035bca0c9a2c2ff1b40e6f4d5299f301aa2eb525b93cfa5e9084fd1e', 'label': '/data', 'dir': '/data', 'keepfiles': True, 'keepsnaphot': False}], 'dirs': [{'label': '/results', 'dir': '/results', 'keepfiles': True, 'onconflicts': {'onstart': {'keeplocal': False}, 'onclose': {'keeplocal': True}}}]}, 'maxwaittime': -1, 'maxexectime': 600, 'maxretries': 3}
